In [1]:
import requests
import os
import shutil

C:\Users\Rono\anaconda3\envs\learn-env\lib\site-packages\requests\__init__.py:89: RequestsDependencyWarning: urllib3 (2.1.0) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [2]:
# years we want to scrape
years = list(range(1991,2022))

In [3]:
# loop that saves html files to mvp folder
url_start = "https://www.basketball-reference.com/awards/awards_{}.html"

for year in years:
    url = url_start.format(year)
    
    data = requests.get(url)
    
    with open("mvp/{}.html".format(year), "w+", encoding= "utf-8") as f:
        f.write(data.text)

### Parsing mvp table with bsp 

In [4]:
from bs4 import BeautifulSoup

In [6]:
with open("mvp/1991.html", encoding='utf-8') as f:
    page = f.read()
    
soup = BeautifulSoup(page, 'html.parser')
soup.find('tr', class_="over_header").decompose()

In [7]:
mvp_table = soup.find_all(id="mvp")[0]
#mvp_table

In [8]:
import pandas as pd

#### Read HTML into pandas 

In [9]:
mvp_1991 = pd.read_html(str(mvp_table))[0]
mvp_1991.head(2)

,Rank,Player,Age,Tm,First,Pts Won,Pts Max,Share,G,MP,PTS,TRB,AST,STL,BLK,FG%,3P%,FT%,WS,WS/48
0,1,Michael Jordan,27,CHI,77.0,891.0,960,0.928,82,37.0,31.5,6.0,5.5,2.7,1.0,0.539,0.312,0.851,20.3,0.321
1,2,Magic Johnson,31,LAL,10.0,497.0,960,0.518,79,37.1,19.4,7.0,12.5,1.3,0.2,0.477,0.320,0.906,15.4,0.251


In [10]:
mvp_1991["Year"] = 1991

In [11]:
mvp_1991.head(3)

,Rank,Player,Age,Tm,First,Pts Won,Pts Max,Share,G,MP,...,TRB,AST,STL,BLK,FG%,3P%,FT%,WS,WS/48,Year
0,1,Michael Jordan,27,CHI,77.0,891.0,960,0.928,82,37.0,...,6.0,5.5,2.7,1.0,0.539,0.312,0.851,20.3,0.321,1991
1,2,Magic Johnson,31,LAL,10.0,497.0,960,0.518,79,37.1,...,7.0,12.5,1.3,0.2,0.477,0.320,0.906,15.4,0.251,1991
2,3,David Robinson,25,SAS,6.0,476.0,960,0.496,82,37.7,...,13.0,2.5,1.5,3.9,0.552,0.143,0.762,17.0,0.264,1991


In [12]:
# loop to create multiple dfs for all years
dfs = []
for year in years:
    with open("mvp/{}.html".format(year), encoding='utf-8') as f:
        page = f.read()
    
    soup = BeautifulSoup(page, 'html.parser')
    soup.find('tr', class_="over_header").decompose()
    mvp_table = soup.find_all(id="mvp")[0]
    mvp_df = pd.read_html(str(mvp_table))[0]
    mvp_df["Year"] = year        # will help us know which year mvp is from
    dfs.append(mvp_df)

In [13]:
# combine all these dfs into 1
mvps = pd.concat(dfs)

mvps.tail()

,Rank,Player,Age,Tm,First,Pts Won,Pts Max,Share,G,MP,...,TRB,AST,STL,BLK,FG%,3P%,FT%,WS,WS/48,Year
10,11,Russell Westbrook,32,WAS,0.0,5.0,1010,0.005,65,36.4,...,11.5,11.7,1.4,0.4,0.439,0.315,0.656,3.7,0.075,2021
11,12,Ben Simmons,24,PHI,0.0,3.0,1010,0.003,58,32.4,...,7.2,6.9,1.6,0.6,0.557,0.300,0.613,6.0,0.153,2021
12,13T,James Harden,31,TOT,0.0,1.0,1010,0.001,44,36.6,...,7.9,10.8,1.2,0.8,0.466,0.362,0.861,7.0,0.208,2021
13,13T,LeBron James,36,LAL,0.0,1.0,1010,0.001,45,33.4,...,7.7,7.8,1.1,0.6,0.513,0.365,0.698,5.6,0.179,2021
14,13T,Kawhi Leonard,29,LAC,0.0,1.0,1010,0.001,52,34.1,...,6.5,5.2,1.6,0.4,0.512,0.398,0.885,8.8,0.238,2021


In [14]:
mvps.to_csv('mvps.csv')

# Now getting player stats

In [15]:
# rewriting above code to work for players url
player_stats_url = "https://www.basketball-reference.com/leagues/NBA_{}_per_game.html"

for year in years:
    url = player_stats_url.format(year)
    
    data = requests.get(url)
    
    with open("player/{}.html".format(year), "w+", encoding='utf-8') as f:
        f.write(data.text)

But if you check in the player folder, the stored htmls have only 17 players, while the original site has over 600 player stats. This is because the site is in java and is causing rendering problems. It assumes my web renders the page in a browser but thats not the case.

So trying to find a way to use my web to render java. 

# Selenium

In [16]:
import os
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time

In [17]:
driver = webdriver.Chrome(
    executable_path="C:/Users/Rono/Downloads/chrome-win64"
    )


TypeError: __init__() got an unexpected keyword argument 'executable_path'

In [18]:
dfs = []
for year in years:
    with open("player/{}.html".format(year), encoding='utf-8') as f:
        page = f.read()
    
    soup = BeautifulSoup(page, 'html.parser')
    soup.find('tr', class_="thead").decompose()
    player_table = soup.find_all(id="per_game_stats")[0]
    player_df = pd.read_html(str(player_table))[0]
    player_df["Year"] = year
    dfs.append(player_df)

AttributeError: 'NoneType' object has no attribute 'decompose'

In [ ]:
players = pd.concat(dfs)

In [ ]:
players

In [ ]:
players.to_csv('players.csv')

# Division standings

In [ ]:
team_stats_url = "https://www.basketball-reference.com/leagues/NBA_{}_standings.html"

In [ ]:
# demo of single year
year = 1991
url = team_stats_url.format(year)

data = requests.get(url)

with open("team/{}.html".format(year), "w+", encoding='utf-8') as f:
    f.write(data.text)

In [ ]:
# looping for all years
for year in years:
    url = team_stats_url.format(year)
    
    data = requests.get(url)
    
    with open("team/{}.html".format(year), "w+", encoding='utf-8') as f:
        f.write(data.text)

In [ ]:
dfs = []
for year in years:
    with open("team/{}.html".format(year), encoding='utf-8') as f:
        page = f.read()
    
    soup = BeautifulSoup(page, 'html.parser')
    #soup.find('tr', class_="thead").decompose()
    e_table = soup.find_all(id="divs_standings_E")
    e_df = pd.read_html(str(e_table))[0]
    e_df["Year"] = year
    e_df["Team"] = e_df["Eastern Conference"]
    del e_df["Eastern Conference"]
    dfs.append(e_df)
    
    w_table = soup.find_all(id="divs_standings_W")
    w_df = pd.read_html(str(w_table))[0]
    w_df["Year"] = year
    w_df["Team"] = w_df["Western Conference"]
    del w_df["Western Conference"]
    dfs.append(w_df)

In [ ]:
teams = pd.concat(dfs)

In [ ]:
teams.to_csv("teams.csv")